Example for Databricks-embedded Data Quality Framework DQX

In [0]:
%python
import yaml 
from databricks.labs.dqx import check_funcs

from databricks.labs.dqx.config import InputConfig, OutputConfig
from databricks.labs.dqx.engine import DQEngine
from databricks.labs.dqx.rule import DQRowRule

checks = yaml.safe_load("""
- name: trip_distance_positive
  criticality: error
  check:
    function: sql_expression
    arguments:
      expression: "trip_distance > 0"

- name: passenger_count_valid
  criticality: error
  check:
    function: sql_expression
    arguments:
      expression: "passenger_count >= 1 and passenger_count <= 6"

- name: payment_type_allowed
  criticality: error
  check:
    function: is_in_list
    arguments:
      column: payment_type
      allowed:
        - 1
        - 2
        - 3
        - 4
        - 5
""")


from databricks.sdk import WorkspaceClient

# Initialize DQEngine
dq_engine = DQEngine(WorkspaceClient())
status = DQEngine.validate_checks(checks)
print(status)


# Apply checks in-memory
valid_df, invalid_df = dq_engine.apply_checks_and_split(df_month, checks)
